# EXP1: Prever semanas de fevereiro usando dados de janeiro 

In [102]:
import pandas as pd

In [148]:
full_df = pd.read_parquet("/data/challenges/hackathon_bigdata_2025/processed_data/transacoes_merged.parquet")
target_df = pd.read_parquet("/data/challenges/hackathon_bigdata_2025/processed_data/targets.parquet")

In [149]:
target_df.head()

,semana,pdv,produto,quantidade,mes,ano
0,1,1001371918471115422,1029370090212151375,2.0,1,2022
1,1,1001371918471115422,1120490062981954254,18.0,1,2022
2,1,1001371918471115422,2239307647969388381,2.0,1,2022
3,1,1001371918471115422,4353552881410365573,7.0,1,2022
4,1,1001371918471115422,4797439216678436447,1.0,1,2022


In [150]:
# Identificar as combinações de PDV+produto que estão em ambos os dataframes
full_df['pdv_produto'] = full_df['pdv'].astype(str) + '_' + full_df['produto'].astype(str)
target_df['pdv_produto'] = target_df['pdv'].astype(str) + '_' + target_df['produto'].astype(str)

In [151]:
(target_df.quantidade == 0).sum(), (target_df.quantidade < 0).sum()

(np.int64(22965), np.int64(54232))

In [152]:
target_train = target_df[target_df['mes'] == 4]
target_val = target_df[target_df['mes'] == 5]

In [153]:
x_df_train = full_df[full_df['mes'] == 3]
x_df_val = full_df[full_df['mes'] == 4]

In [154]:
target_train.shape, x_df_train.shape, x_df_val.shape, target_val.shape,

((459469, 7), (493306, 21), (459469, 21), (541797, 7))

In [155]:
target_train.groupby('pdv').value_counts()

pdv                  semana  produto              quantidade  mes  ano   pdv_produto                            
1000237487041964405  4       4038588102284338370  1.0         4    2022  1000237487041964405_4038588102284338370    1
                             596381974901127871   1.0         4    2022  1000237487041964405_596381974901127871     1
                             7270233133454638680  1.0         4    2022  1000237487041964405_7270233133454638680    1
                             7370044109082767116  1.0         4    2022  1000237487041964405_7370044109082767116    1
                             777251454728290683   2.0         4    2022  1000237487041964405_777251454728290683     1
                                                                                                                   ..
997907800111849739   4       8054344730735101996  2.0         4    2022  997907800111849739_8054344730735101996     1
                             8222093130128661995  5.0        

In [156]:
target_train = target_train[target_train['pdv_produto'].isin(x_df_train['pdv_produto'])]

In [157]:
full_df.marca.nunique(), full_df.categoria_pdv.nunique(), full_df.categoria.nunique(), full_df.marca.nunique(), full_df.tipos.nunique(), full_df.label.nunique()

(4209, 54, 7, 4209, 22, 14)

In [158]:
full_df.categoria.unique()

array(['Package', 'Non-Alcohol', 'ABA Spirits', 'Draft', 'Wine',
       'Distilled Spirits', 'Tobacco'], dtype=object)

In [159]:
full_df.tipos.unique()

array(['Package', 'Non Alcohol', 'Wine > 14 %', 'Draft', 'Wine < 14 %',
       'N/A Beer', 'Distilled Spirits-RTD', 'Distilled Spirits',
       'Talking Rain', 'Wine RTD', 'Cider Pkg', 'Wine < 14 % - Domestic',
       'Hispanic', 'Allocated Spirits', 'Distilled Spirits-Domest',
       'Dist Spirits-RTD Domestic', 'MB Wine < 14 %', 'Cider Dft',
       'Dairy', 'Non Alcohol W&S', 'Tobacco', 'Pkg 3.2'], dtype=object)

In [160]:
full_df.categoria_pdv.unique()

array(['Convenience', 'Package/Liquor', 'Irish', 'Restaurant', 'Theatre',
       'Super Center', 'Grocery', 'Bar', 'Barbeque', 'Mexican Rest',
       'Italian', 'Billiard/Bowling', 'Service Org', 'Pizza', 'Asian',
       'Church', 'Sports/Rec Club', 'Hotel/Motel', 'Stadium/Concession',
       'Special Event', 'Drug', 'Banquet/Caterer', 'Club Store',
       'Airline/Airport', 'All Other N/A On Premise', 'Sample Room',
       'Night Club', 'Golf - Public', 'Coffee House', 'Bodega',
       'Neighborhood Store', 'Golf - Private', 'Salon/Spa/Tann',
       'Adult Entertainment', 'Music Venue', 'Gay Bar', 'Korean',
       'Military', 'Sub Distributor', 'Other On Premise', 'Theme Park',
       'Winery', 'Country/Western', 'Other Off Premise',
       'Non-Traditional', 'Race Track', 'French',
       'All Other N/A Off Premise', 'Gym/Fitness', 'German', 'Casino',
       'Health Club', 'Marina / Lake', 'Mass Merch'], dtype=object)

In [161]:
full_df.label.unique()

array(['Core', 'Discontinued', 'In&Out', None, 'Close Out', 'Specialty',
       'Winter Seasonal', 'Fall Seasonal', 'Private Label',
       'Summer Seasonal', 'Spring Seasonal', 'Speciality', 'Clearing',
       'Allocated', 'Other'], dtype=object)

In [162]:
full_df.marca.unique()

array(['Michelob Ultra', 'Bud Light', 'Natural Light', ...,
       'Echo de Lynch Bages', 'Chateau Grand Barrail Lamarzelle Figeac',
       'Wicked Weed Perni-Haze Hazy IPA'], shape=(4209,), dtype=object)

In [163]:
full_df.iloc[0]

pdv                                  1001371918471115422
zipcode                                            30175
produto                              1029370090212151375
semana                                                 1
mes                                                    1
ano                                                 2022
categoria_pdv                                Convenience
categoria                                        Package
premise                                      Off Premise
tipos                                            Package
label                                               Core
marca                                     Michelob Ultra
fabricante                         AB Anheuser Busch Inc
descricao                      MICHELOB ULTRA 2/12/12 CN
gross_value                                    54.299999
net_value                                      49.740486
gross_profit                                   16.224486
discount                       

In [164]:
# Candidatos a features:
# Categóricas:
# - label
# - categoria_pdv
# - categoria
# - marca (1940 casos)

# Númericas:
# quantidade
# discount

# Temporais:


In [218]:
# Select only pdv_produto rows where in target_df there are more than 4 entries

values = (target_train.pdv_produto.value_counts() >= 4).values
filtered_target_train = target_train[target_train['pdv_produto'].isin(target_train.pdv_produto.value_counts()[values].index)]

values = (target_val.pdv_produto.value_counts() >= 4).values
filtered_target_val = target_val[target_val['pdv_produto'].isin(target_val.pdv_produto.value_counts()[values].index)]

In [219]:
filtered_target_train.shape, filtered_target_val.shape

((64405, 7), (100935, 7))

In [220]:
filtered_target_train_week_1 = filtered_target_train[filtered_target_train['semana'] == 1]
filtered_target_train_week_1.pdv_produto.nunique()

filtered_target_val_week_1 = filtered_target_val[filtered_target_val['semana'] == 1]
filtered_target_train_week_1.pdv_produto.nunique(), filtered_target_val_week_1.pdv_produto.nunique()

(15306, 22251)

In [221]:
filtered_X_train = x_df_train[x_df_train['pdv_produto'].isin(filtered_target_train_week_1['pdv_produto'])]
filtered_X_train.shape
filtered_X_train.pdv_produto.nunique()

15306

In [222]:
filtered_X_val = x_df_val[x_df_val['pdv_produto'].isin(filtered_target_val['pdv_produto'])]
filtered_X_val.shape
filtered_X_val.pdv_produto.nunique()

23126

In [223]:
filtered_X_train;

In [224]:
def aggr_df(df):
    # In filtered X, create features such as "quantidade_semana_1", "quantidade_semana_2", "quantidade_semana_3", "quantidade_semana_4" for each pdv_produto value
    filtered_df = df.copy()
    filtered_X_agg = filtered_df.pivot_table(index='pdv_produto', columns='semana', values='quantidade', aggfunc='sum')
    filtered_X_agg.columns = [f'quantidade_semana_{col}' for col in filtered_X_agg.columns]
    filtered_X_agg = filtered_X_agg.reset_index()
    # Also create features discount_semana_1, discount_semana_2, discount_semana_3, discount_semana_4
    discount_agg = filtered_df.pivot_table(index='pdv_produto', columns='semana', values='discount', aggfunc='sum')
    discount_agg.columns = [f'discount_semana_{col}' for col in discount_agg.columns]
    discount_agg = discount_agg.reset_index()
    # For categorical features, create features such as "label", "categoria_pdv", "categoria" using the first value found for that pdv_produto
    categorical_agg = filtered_X.groupby('pdv_produto').agg({
        'label': 'first',
        'categoria_pdv': 'first',
        'categoria': 'first',
        #'marca': 'first',
        #'tipos': 'first'
    }).reset_index()
    # Merge all features into a single dataframe
    filtered_X_agg = filtered_X_agg.merge(discount_agg, on='pdv_produto', how='left')
    filtered_X_agg = filtered_X_agg.merge(categorical_agg, on='pdv_produto', how='left')
    filtered_X_agg = filtered_X_agg.fillna(0)

    # For each categorical feature, perform one-hot encoding (label, categoria_pdv, categoria, tipos)
    categorical_features = ['label', 'categoria_pdv', 'categoria']
    filtered_X_agg = pd.get_dummies(filtered_X_agg, columns=categorical_features, drop_first=True)

    return filtered_X_agg

filtered_train = aggr_df(filtered_X_train)
filtered_val = aggr_df(filtered_X_val)

In [225]:
train_week1 = filtered_train.merge(filtered_target_train_week_1[['pdv_produto', 'quantidade']], on='pdv_produto', how='left')
val_week1 = filtered_val.merge(filtered_target_val_week_1[['pdv_produto', 'quantidade']], on='pdv_produto', how='left')

In [226]:
X1_train = train_week1.drop(columns=['pdv_produto', 'quantidade'])
y1_train = train_week1['quantidade']

X1_val = val_week1.drop(columns=['pdv_produto', 'quantidade'])
y1_val = val_week1['quantidade']

In [227]:
# Replace any blank space in columns with _
#X.columns = X.columns.str.replace(' ', '_')

In [228]:
X1_train.columns.shape, y1_train.shape, X1_val.shape, y1_val.shape

((46,), (15306,), (23126, 48), (23126,))

In [229]:
# Create X1_train columns that are missing and are in X1_val
for col in X1_val.columns:
    if col not in X1_train.columns:
        X1_train[col] = 0

for col in X1_train.columns:
    if col not in X1_val.columns:
        X1_val[col] = 0

In [230]:
X1_train.columns.shape, X1_val.columns.shape, y1_val.shape

((49,), (49,), (23126,))

In [265]:
import lightgbm as lgb
import numpy as np

def wmape_metric(y_true, y_pred):
    """
    Calcula o Weighted Mean Absolute Percentage Error (wMAPE) para o LightGBM.
    A função precisa retornar uma lista de tuplas no formato:
    (nome_da_metrica, valor_da_metrica, is_higher_better)
    """
    sum_of_true_values = np.sum(np.abs(y_true))
    if sum_of_true_values == 0:
        return [('wmape', 0.0, False)] # Se não há vendas, o erro é zero.

    wmape_score = np.sum(np.abs(y_true - y_pred)) / sum_of_true_values
    
    return [('wmape', wmape_score, False)]

lgbm = lgb.LGBMRegressor(
    objective='regression',
    metric=None,
    n_estimators=100,
    learning_rate=0.05,
    random_state=41,
    #max_depth=50,
    num_leaves=10,
    min_child_samples=10,
    reg_alpha=0.01,
    reg_lambda=0.01,
)

lgbm.fit(
    X1_train, 
    y1_train,
    eval_metric=wmape_metric,
    eval_set=[(X1_val, y1_val)],
    callbacks=[lgb.early_stopping(1000, verbose=True)], # Para o treino se a métrica não melhorar em 200 rodadas
    #categorical_feature=['pdv_id', 'product_id'] # Informa quais colunas são categóricas
)

# --- 4. Previsão e Avaliação ---
y_pred_train = lgbm.predict(X1_train)
y_pred = lgbm.predict(X1_val)

wmape_train = np.sum(np.abs(y1_train - y_pred_train)) / np.sum(np.abs(y1_train))
print(f"WMAPE Train - Week 1: {wmape_train:.4f}")

wmape_val = np.sum(np.abs(y1_val - y_pred)) / np.sum(np.abs(y1_val))
print(f"WMAPE Val - Week 1: {wmape_val:.4f}")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000792 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1612
[LightGBM] [Info] Number of data points in the train set: 15306, number of used features: 30
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 4.571478
Training until validation scores don't improve for 1000 rounds
Did not meet early stopping. Best iteration is:
[97]	valid_0's l2: 18.9312	valid_0's wmape: 0.458128
WMAPE Train - Week 1: 0.3599
WMAPE Val - Week 1: 0.3951


In [262]:
filtered_target_train_week_2 = filtered_target_train[filtered_target_train['semana'] == 2]
filtered_target_val_week_2 = filtered_target_val[filtered_target_val['semana'] == 2]

filtered_X_train = x_df_train[x_df_train['pdv_produto'].isin(filtered_target_train_week_2['pdv_produto'])]
filtered_X_val = x_df_val[x_df_val['pdv_produto'].isin(filtered_target_val_week_2['pdv_produto'])]

filtered_train = aggr_df(filtered_X_train)
filtered_val = aggr_df(filtered_X_val)

train_week2 = filtered_train.merge(filtered_target_train_week_2[['pdv_produto', 'quantidade']], on='pdv_produto', how='left')
val_week2 = filtered_val.merge(filtered_target_val_week_2[['pdv_produto', 'quantidade']], on='pdv_produto', how='left')

X2_train = train_week2.drop(columns=['pdv_produto', 'quantidade'])
y2_train = train_week2['quantidade']

X2_val = val_week2.drop(columns=['pdv_produto', 'quantidade'])
y2_val = val_week2['quantidade']

# Create X2_train columns that are missing and are in X2_val
for col in X2_val.columns:
    if col not in X2_train.columns:
        X2_train[col] = 0

for col in X2_train.columns:
    if col not in X2_val.columns:
        X2_val[col] = 0

In [266]:
import lightgbm as lgb
import numpy as np

lgbm = lgb.LGBMRegressor(
    objective='regression',
    metric=None,
    n_estimators=200,
    learning_rate=0.05,
    random_state=41,
    #max_depth=50,
    num_leaves=10,
    min_child_samples=10,
    reg_alpha=0.01,
    reg_lambda=0.01,
)

lgbm.fit(
    X2_train, 
    y2_train,
    eval_metric=wmape_metric,
    eval_set=[(X2_val, y2_val)],
    callbacks=[lgb.early_stopping(1000, verbose=True)], # Para o treino se a métrica não melhorar em 200 rodadas
    #categorical_feature=['pdv_id', 'product_id'] # Informa quais colunas são categóricas
)

# --- 4. Previsão e Avaliação ---
y_pred_train = lgbm.predict(X2_train)
y_pred = lgbm.predict(X2_val)

wmape_train = np.sum(np.abs(y2_train - y_pred_train)) / np.sum(np.abs(y2_train))
print(f"WMAPE Train - Week 2: {wmape_train:.4f}")

wmape_val = np.sum(np.abs(y2_val - y_pred)) / np.sum(np.abs(y2_val))
print(f"WMAPE Val - Week 2: {wmape_val:.4f}")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000755 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1615
[LightGBM] [Info] Number of data points in the train set: 15233, number of used features: 30
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 4.558111
Training until validation scores don't improve for 1000 rounds
Did not meet early stopping. Best iteration is:
[129]	valid_0's l2: 17.1829	valid_0's wmape: 0.401728
WMAPE Train - Week 2: 0.3657
WMAPE Val - Week 2: 0.4017
